In [35]:
import ROOT
import array as array
from array import array 
import os
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
#import root_numpy as rtnpy
#from root_numpy import *
import h5py
#from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
import sys
sys.path.insert(0,'../test/')
from variables_WSel import nSubVariables_WSel
from variables_topSel import nSubVariables_topSel
#from DrawHistogram_Wtop import *
#from Samples_WtopSel import checkDict, dictSamples
sys.path.insert(0,'../python/')
import CMS_lumi as CMS_lumi
import tdrstyle as tdrstyle


In [37]:
%jsroot on

In [38]:
def getListOfFiles(dirName, histos=False):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    #To ensure event counts don't come up against  a histogram file and rather just the nanoskim
    if histos==False: listOfFile[:] = [x for x in listOfFile if "histograms" not in x]
    #print (listOfFile)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):            
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

#print (getListOfFiles('/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0000/'))

In [39]:
#!/usr/bin/env python
### For xsec calculation: grep miniAOD datasets.py | awk '{ print $4 }' | sed "s/'//g" | sed  's/"//g' | sed 's/\,//g' > calculateXSectionAndFilterEfficiency/datasets.txt
import ROOT

dictSamples = {
    
    'SingleMuon' : {
        
        '2017' :  {
            'skimmerHisto' : 'SingleMuonAll_2017UL.root',
            'lumi' : 41472.052,
            },
        '2018' :  {
           'skimmerHisto' : 'SingleMuonAll_2018UL.root',
            'lumi' : 59816.0#56385.82159,
            },
        'all' :  {
           'skimmerHisto' : 'SingleMuonAll_UL17-18.root',
           'lumi' : 59816.0#56385.82159,
            },
        'color': ROOT.kWhite,
        'selection' : 'Wtop'
    },

    'TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'TTJets_amcatnloFXFX-pythia8_2017UL.root',
            'nGenWeights' : 286629789811.7677,
            },
        '2018' :  {
            'skimmerHisto' : 'TTJets_amcatnloFXFX-pythia8_2018UL.root',
            'nGenWeights' : 297902725633.38257,
            },
        
        'all' :  {
            'skimmerHisto' : 'TTJets_amcatnloFXFX-pythia8_UL17-18.root',
            'nGenWeights' : 584532515445.1503,
            },
    
        'XS' : 7.535e+02, #some notes say 722.8 and a few say 831.76?
        'selection' : 'Wtop',
        'label' : 't#bar{t} amcatnloFXFX' ,
        'color': ROOT.kBlue
    },

    'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'TTToSemileptonic_powheg_pythia8_2017UL_NoTopReWt.root',
            'nGenWeights' : 32392787477.907993,
            },
        '2018' :  {
            'skimmerHisto' : 'TTToSemileptonic_powheg_pythia8_2018UL_NoTopReWt.root',
            'nGenWeights' : 31378075292.893986,
            },
        'all' :  {
            'skimmerHisto' : 'TTToSemileptonic_powheg_pythia8_nom_UL17-18_NoTopReWt.root',
            'nGenWeights' : 63770862770.80198,
              },
        'XS' : 301.64,#364.0,#5.34,
        'selection' : 'Wtop',
        'label' : 't#bar{t} powheg' ,
        'color': ROOT.kMagenta
    },

    'TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'TTTo2L2Nu_powheg_pythia8_2017UL.root',
            'nGenWeights' :  4759777104.469413,
            },
        '2018' :  {
            'skimmerHisto' : 'TTTo2L2Nu_powheg_pythia8_2018UL.root',
            'nGenWeights' : 5063754431.9676,
            },
        'all' :  {
            'skimmerHisto' : 'TTTo2L2Nu_powheg_pythia8_UL17-18.root',
            'nGenWeights' : 9823531536.437012,
            },
        'selection' : 'Wtop',
        'XS' : 72.83,#26,
        'label' : 't#bar{t} DL' ,
        'color': ROOT.kCyan+1
    },

    'WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'WJetsToLNu_madgraphMLM_pythia8_2017UL.root',
            'nGenWeights' : 9949891586212.455078,
            },
        '2018' :  {
            'skimmerHisto' : 'WJetsToLNu_madgraphMLM_pythia8_2018UL.root',
            'nGenWeights' : 10484986870276.357422,
            },
        'all' :  {
            'skimmerHisto' : 'WJetsToLNu_madgraphMLM_pythia8_UL17-18.root',
            'nGenWeights' : 20434878456488.812,
            },
        'selection' : 'Wtop',
        'XS' : 5.371e+04,
        'label' : 'WJets' ,
        'color': ROOT.kOrange-2
    },
    'ST_s-channel_4f_leptonDecays_TuneCP5_13TeV-amcatnlo-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'ST_s-channel_amcatnlo_pythia8_2017UL.root',
            'nGenWeights' : 70439807.475972,
            },
        '2018' :  {
            'skimmerHisto' : 'ST_s-channel_amcatnlo_pythia8_2018UL.root',
            'nGenWeights' : 64014403.924644,
            },
        'all' :  {
            'skimmerHisto' : 'ST_s-channel_amcatnlo_pythia8_UL17-18.root',
            'nGenWeights' : 134454211.400616,
            },
        'selection' : 'Wtop',
        'XS' : 3.549,
        'label' : 'Single top' ,
        'color': ROOT.kGreen+2,
    },
    'ST_t-channel_top_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'ST_t-channel_top_powheg_pythia8_2017UL.root',
            'nGenWeights' : 656310119.274000,
            },
        '2018' :  {
            'skimmerHisto' : 'ST_t-channel_top_powheg_pythia8_2018UL.root',
            'nGenWeights' : 683747124.990000,
            },
        'all' :  {
            'skimmerHisto' : 'ST_t-channel_top_powheg_pythia8_UL17-18.root',
            'nGenWeights' : 1340057244.264,
            },
        'selection' : 'Wtop',
        'XS' : 1.197e+02,
        'label' : 'Single top' ,
        'color': ROOT.kGreen+2,
    },
    'ST_t-channel_antitop_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'ST_t-channel_antitop_powheg_pythia8_2017UL.root',
            'nGenWeights' : 303960792.435600,
            },
        '2018' :  {
            'skimmerHisto' : 'ST_t-channel_antitop_powheg_pythia8_2018UL.root',
            'nGenWeights' : 271219283.652000,
            },
        'all' :  {
            'skimmerHisto' : 'ST_t-channel_antitop_powheg_pythia8_UL17-18.root',
            'nGenWeights' : 575180076.0876,
            },
        'selection' : 'Wtop',
        'XS' : 7.174e+01,
        'label' : 'Single top' ,
        'color': ROOT.kGreen+2,
    },
    'ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'ST_tW_top_powheg_pythia8_2017UL.root',
            'nGenWeights' : 331239851.967700,
            },
        '2018' :  {
            'skimmerHisto' : 'ST_tW_top_powheg_pythia8_2018UL.root',
            'nGenWeights' : 328321574.253000,
            },
        'all' :  {
            'skimmerHisto' : 'ST_tW_top_powheg_pythia8_UL17-18.root',
            'nGenWeights' : 659561426.2207,
            },
        'selection' : 'Wtop',
        'XS' : 3.245e+01,
        'label' : 'Single top' ,
        'color': ROOT.kGreen+2,
    },
    'ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'ST_tW_antitop_powheg_pythia8_2017UL.root',
            'nGenWeights' : 303960792.435600,
            },
        '2018' :  {
            'skimmerHisto' : 'ST_tW_antitop_powheg_pythia8_2018UL.root',
            'nGenWeights' : 296655487.557600,
            },
        'all' :  {
            'skimmerHisto' : 'ST_tW_antitop_powheg_pythia8_UL17-18.root',
            'nGenWeights' : 600616279.9932001,
            },
        'selection' : 'Wtop',
        'XS' : 3.251e+01,
        'label' : 'Single top' ,
        'color': ROOT.kGreen+2,
    },
    'WW_TuneCP5_13TeV-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'WW_pythia8_2017UL.root',
            'nGenWeights' : 15634116.199514,
            },
        '2018' :  {
            'skimmerHisto' : 'WW_pythia8_2018UL.root',
            'nGenWeights' : 7959266.140092,
            },
        'all' :  {
            'skimmerHisto' : 'WW_pythia8_UL17-18.root',
            'nGenWeights' : 23593382.339606002,
            },
        'selection' : 'Wtop',
        'XS': 7.56e+01,
        'label' : 'Diboson' ,
        'color': ROOT.kPink-1
    },
    'ZZ_TuneCP5_13TeV-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'ZZ_pythia8_2017UL.root',
            'nGenWeights' : 2706000.000000,
            },
        '2018' :  {
            'skimmerHisto' : 'ZZ_pythia8_2018UL.root',
            'nGenWeights' : 2000000.000000,
            },
        'all' :  {
            'skimmerHisto' : 'ZZ_pythia8_UL17-18.root',
            'nGenWeights' : 4706000.000000,
            },
        'selection' : 'Wtop',
        'XS' : 1.210e+01,
        'label' : 'Diboson' ,
        'color': ROOT.kPink-1
    },
    'WZ_TuneCP5_13TeV-pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'WZ_pythia8_2017UL.root',
            'nGenWeights' : 7889000.000000,
            },
        '2018' :  {
            'skimmerHisto' : 'WZ_pythia8_2018UL.root',
            'nGenWeights' : 3976000.000000,
            },
        'all' :  {
            'skimmerHisto' : 'WZ_pythia8_UL17-18.root',
            'nGenWeights' : 11865000.0,
            },
        'XS' : 2.751e+01,
        'selection' : 'Wtop',
        'label' : 'Dibosons' ,
        'color': ROOT.kPink-1
    },
    'QCD_Pt-170to300_MuEnrichedPt5_TuneCP5_13TeV_pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_170to300_pythia8_2017UL.root',
            'nGenWeights' : 36027673.00,
            },
        '2018' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_170to300_pythia8_2018UL.root',
            'nGenWeights' : 35868369.000000,
            },
        'all' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_170to300_pythia8_UL17-18.root',
            'nGenWeights' : 71896042.0,
            },
        'XS' : 7.014e+03,
        'selection' : 'Wtop',
        'label' : 'QCD' ,
        'color': ROOT.kRed-7
    },
    'QCD_Pt-300to470_MuEnrichedPt5_TuneCP5_13TeV_pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_300to470_pythia8_2017UL.root',
            'nGenWeights' : 494796.07288,
            },
        '2018' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_300to470_pythia8_2018UL.root',
            'nGenWeights' : 49232711.259666,
            },
        'all' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_300to470_pythia8_UL17-18.root',
            'nGenWeights' : 49727507.332546,
            },
        'XS' : 6.205e+02,
        'selection' : 'Wtop',
        'label' : 'QCD' ,
        'color': ROOT.kRed-7
    },
    'QCD_Pt-470to600_MuEnrichedPt5_TuneCP5_13TeV_pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_470to600_pythia8_2017UL.root',
            'nGenWeights' : 517383.713837,
            },
        '2018' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_470to600_pythia8_2018UL.root',
            'nGenWeights' : 20585907.002748,
            },
        'all' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_470to600_pythia8_UL17-18.root',
            'nGenWeights' : 21103290.716585003,
            },
        'XS' : 5.910e+01,
        'selection' : 'Wtop',
        'label' : 'QCD' ,
        'color': ROOT.kRed-7
    },
    'QCD_Pt-600to800_MuEnrichedPt5_TuneCP5_13TeV_pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_600to800_pythia8_2017UL.root',
            'nGenWeights' : 17318812.571925,
            },
        '2018' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_600to800_pythia8_2018UL.root',
            'nGenWeights' : 16983542.512253,
            },
        'all' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_600to800_pythia8_UL17-18.root',
            'nGenWeights' : 34302355.0842,
            },
        'XS' : 1.825e+01,
        'selection' : 'Wtop',
        'label' : 'QCD' ,
        'color': ROOT.kRed-7
    },
    'QCD_Pt-800to1000_MuEnrichedPt5_TuneCP5_13TeV_pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_800to1000_pythia8_2017UL.root',
            'nGenWeights' : 16962615.000000,
            },
        '2018' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_800to1000_pythia8_2018UL.root',
            'nGenWeights' : 17178227.000000,
            },
        'all' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_800to1000_pythia8_UL17-18.root',
            'nGenWeights' : 34140842.0,
            },
        'XS' : 3.287e+00,
        'selection' : 'Wtop',
        'label' : 'QCD' ,
        'color': ROOT.kRed-7
    },
    'QCD_Pt-1000toInf_MuEnrichedPt5_TuneCP5_13TeV_pythia8' : {
        '2017' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_1000toInf_pythia8_2017UL.root',
            'nGenWeights' : 14642553.00,
            },
        '2018' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_1000toInf_pythia8_2018UL.root',
            'nGenWeights' : 14556168.000000,
            },
        'all' :  {
            'skimmerHisto' : 'QCD_Muen_Pt_1000toInf_pythia8_UL17-18.root',
            'nGenWeights' : 29198721,
            },
        'XS' : 1.087e+00,
        'selection' : 'Wtop',
        'label' : 'QCD' ,
        'color': ROOT.kRed-7
    },    
}


def checkDict( string, dictio ):
    return next(v for k,v in dictio.items() if string.startswith(k))



### Combined UL17 + UL18

In [40]:
import array
from array import array
def Bin(Range):
    bins=np.array([x for x in np.linspace(num=Range[0]+1,start=Range[1],stop=Range[2])])
    return bins

def getRebinnedRescaledHist(hist_name, file_name, xrange, lumi, xsec, nevt, isMC=True, rebin=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    if rebin: newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    else: newHist = hist.Clone()
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(lumi*xsec/nevt)
    
    return newHist

In [41]:
Hists_WSel=np.array([
    'recoJet_pt_nom_WSel', 'recoJet_mass_nom_WSel', 'recoJet_tau21_nom_WSel', 'recoJet_tau32_nom_WSel',
    'recoJet_tau_0p5_1_nom_WSel', 'recoJet_tau_1_1_nom_WSel', 'recoJet_tau_2_1_nom_WSel',
    'recoJet_tau_0p5_2_nom_WSel', 'recoJet_tau_1_2_nom_WSel', 'recoJet_tau_2_2_nom_WSel',
    'recoJet_tau_0p5_3_nom_WSel', 'recoJet_tau_1_3_nom_WSel', 'recoJet_tau_2_3_nom_WSel',
    'recoJet_tau_0p5_4_nom_WSel', 'recoJet_tau_1_4_nom_WSel', 'recoJet_tau_2_4_nom_WSel',
    'nPVs_WSel','nleps_WSel',
    'muons_pt_WSel','muons_eta_WSel','muons_phi_WSel',
    'nAK8jets_WSel','AK8jets_pt_WSel','AK8jets_eta_WSel','AK8jets_phi_WSel','AK8jets_mass_WSel',
    'nAK4jets_WSel','AK4jets_pt_WSel','AK4jets_eta_WSel','AK4jets_phi_WSel','AK4jets_mass_WSel',
    'METPt_WSel','HT_WSel'
    ])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events/(50 GeV)','P_{t}','P_{t}/GeV'],['Events/(10 GeV)','Softdrop Mass','Softdrop Mass/GeV'],['Events/0.05','#tau_{21}','#tau_{21}'],['Events/0.05','#tau_{32}','#tau_{32}'],
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    ['Events/2','Number of PV','Number of PV'],['Events','Number of Leptons','Number of Leptons'],
    ['Events/(20 GeV)','P_{t} of Muons','P_{t} of Muons/GeV'],['Events/0.4','#eta of Muons','#eta of Muons'],['Events/(#pi/10)','#phi of Muons','#phi of Muons'],
    ['Events','Number of AK8Jets','Number of AK8Jets'],['Events/(60 GeV)','P_{t} of AK8Jets','P_{t} of AK8Jets/GeV'],['Events/0.4','#eta of AK8Jets','#eta of AK8Jets'],['Events/(#pi/10)','#phi of AK8Jets','#phi of AK8Jets'],['Events/(10 GeV)','Mass of AK8Jets','Mass of AK8Jets/GeV'],
    ['Events','Number of AK4Jets','Number of AK4Jets'],['Events/(40 GeV)','P_{t} of AK4Jets','P_{t} of AK4Jets/GeV'],['Events/0.4','#eta of AK4Jets','#eta of AK4Jets'],['Events/(#pi/10)','#phi of AK4Jets','#phi of AK4Jets'],['Events/(20 GeV)','Mass of AK4Jets','Mass of AK4Jets/GeV'],
    ['Events/(20 GeV)','MET P_{t}','MET P_{t}/GeV'],['Events/(100 GeV)','HT','HT/GeV']
    ])
    
Range_WSel=[
    [20,0,1000], [11,50,160],[20,0,1],[20,0,1],
    [30,0,0.9],[35,0,0.7],[20,0,0.4],
    [20,0,0.6],[20,0,0.4],[20,0,0.2],
    [25,0,0.5],[20,0,0.4],[20,0,0.1],
    [20,0,0.4],[20,0,0.2],[20,0,0.1],  
    [35,0,70],[4,0,4],
    [20,0,400],[20,-4,4],[20,-3.14259, 3.14159],
    [8,0,8],[20,0,1200],[20,-4,4],[20,-3.14259, 3.14159],[15,0,150],
    [8,0,8],[20,0,800],[20,-4,4],[20,-3.14259, 3.14159],[20,0,400],
    [30,0,600],[20,0,2000]
]
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

Hists_topSel=np.array([
    'recoJet_pt_nom_topSel', 'recoJet_mass_nom_topSel', 'recoJet_tau21_nom_topSel', 'recoJet_tau32_nom_topSel',
    'recoJet_tau_0p5_1_nom_topSel', 'recoJet_tau_1_1_nom_topSel', 'recoJet_tau_2_1_nom_topSel',
    'recoJet_tau_0p5_2_nom_topSel', 'recoJet_tau_1_2_nom_topSel', 'recoJet_tau_2_2_nom_topSel',
    'recoJet_tau_0p5_3_nom_topSel', 'recoJet_tau_1_3_nom_topSel', 'recoJet_tau_2_3_nom_topSel',
    'recoJet_tau_0p5_4_nom_topSel', 'recoJet_tau_1_4_nom_topSel', 'recoJet_tau_2_4_nom_topSel',
    'nPVs_topSel','nleps_topSel',
    'muons_pt_topSel','muons_eta_topSel','muons_phi_topSel',
    'nAK8jets_topSel','AK8jets_pt_topSel','AK8jets_eta_topSel','AK8jets_phi_topSel','AK8jets_mass_topSel',
    'nAK4jets_topSel','AK4jets_pt_topSel','AK4jets_eta_topSel','AK4jets_phi_topSel','AK4jets_mass_topSel',
    'METPt_topSel','HT_topSel'])


Histnames_topSel=np.array([
    ['Events/(50 GeV)','P_{t}','P_{t}/GeV'],['Events/(10 GeV)','Softdrop Mass','Softdrop Mass/GeV'],['Events/0.05','#tau_{21}','#tau_{21}'],['Events/0.05','#tau_{32}','#tau_{32}'],
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    ['Events/2','Number of PV','Number of PV'],['Events','Number of Leptons','Number of Leptons'],
    ['Events/(20 GeV)','P_{t} of Muons','P_{t} of Muons/GeV'],['Events/0.4','#eta of Muons','#eta of Muons'],['Events/(#pi/10)','#phi of Muons','#phi of Muons'],
    ['Events','Number of AK8Jets','Number of AK8Jets'],['Events/(60 GeV)','P_{t} of AK8Jets','P_{t} of AK8Jets/GeV'],['Events/0.4','#eta of AK8Jets','#eta of AK8Jets'],['Events/(#pi/10)','#phi of AK8Jets','#phi of AK8Jets'],['Events/(10 GeV)','Mass of AK8Jets','Mass of AK8Jets/GeV'],
    ['Events','Number of AK4Jets','Number of AK4Jets'],['Events/(40 GeV)','P_{t} of AK4Jets','P_{t} of AK4Jets/GeV'],['Events/0.4','#eta of AK4Jets','#eta of AK4Jets'],['Events/(#pi/10)','#phi of AK4Jets','#phi of AK4Jets'],['Events/(20 GeV)','Mass of AK4Jets','Mass of AK4Jets/GeV'],
    ['Events/(20 GeV)','MET P_{t}','MET P_{t}/GeV'],['Events/(100 GeV)','HT','HT/GeV']])
    
Range_topSel=[
    [30,0,1500], [12,130,250],[20,0,1],[20,0,1],
    [30,0,0.9],[35,0,0.7],[20,0,0.4],
    [20,0,0.6],[20,0,0.4],[20,0,0.2],
    [25,0,0.5],[20,0,0.4],[20,0,0.1],
    [20,0,0.4],[20,0,0.2],[20,0,0.1],
    
    #[25,-5.,5.],[25,-5.,5.],[25,-5.,5.],
    #[25,-5.,5.],[25,-5.,5.],[25,-5.,5.],
    #[25,-5.,5.],[25,-5.,5.],[25,-5.,5.],
    #[25,-5.,5.],[25,-5.,5.],[25,-5.,5.],
    
    [35,0,70],[4,0,4],
    [20,0,400],[20,-4,4],[20,-3.14259, 3.14159],
    [8,0,8],[20,0,1200],[20,-4,4],[20,-3.14259, 3.14159],[15,0,150],
    [8,0,8],[20,0,800],[20,-4,4],[20,-3.14259, 3.14159],[20,0,400],
    [30,0,600],[20,0,2000]
]
print (len(Hists_topSel),len(Histnames_topSel),len(Range_topSel))

33 33 33
33 33 33


###  all

In [42]:
lumi=41472.052+59816.39


In [43]:
xsecs = list(dictSamples[s]['XS'] for s in list(dictSamples.keys())[1:] )
nevts = list(dictSamples[s]['all']['nGenWeights'] for s in list(dictSamples.keys())[1:] )

fpathsMC = list('../test/Samples/'+dictSamples[s]['all']['skimmerHisto'] for s in list(dictSamples.keys())[1:] )
fpathData = '../test/Samples/'+dictSamples['SingleMuon']['all']['skimmerHisto']
for i in range(0,len(xsecs)):
    print (i, xsecs[i], nevts[i], fpathsMC[i])
print (fpathData)

0 753.5 584532515445.1503 ../test/Samples/TTJets_amcatnloFXFX-pythia8_UL17-18.root
1 301.64 63770862770.80198 ../test/Samples/TTToSemileptonic_powheg_pythia8_nom_UL17-18_NoTopReWt.root
2 72.83 9823531536.437012 ../test/Samples/TTTo2L2Nu_powheg_pythia8_UL17-18.root
3 53710.0 20434878456488.812 ../test/Samples/WJetsToLNu_madgraphMLM_pythia8_UL17-18.root
4 3.549 134454211.400616 ../test/Samples/ST_s-channel_amcatnlo_pythia8_UL17-18.root
5 119.7 1340057244.264 ../test/Samples/ST_t-channel_top_powheg_pythia8_UL17-18.root
6 71.74 575180076.0876 ../test/Samples/ST_t-channel_antitop_powheg_pythia8_UL17-18.root
7 32.45 659561426.2207 ../test/Samples/ST_tW_top_powheg_pythia8_UL17-18.root
8 32.51 600616279.9932001 ../test/Samples/ST_tW_antitop_powheg_pythia8_UL17-18.root
9 75.6 23593382.339606002 ../test/Samples/WW_pythia8_UL17-18.root
10 12.1 4706000.0 ../test/Samples/ZZ_pythia8_UL17-18.root
11 27.51 11865000.0 ../test/Samples/WZ_pythia8_UL17-18.root
12 7014.0 71896042.0 ../test/Samples/QCD_Muen

In [44]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.14

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [45]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 750; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.207, 1., 1., -1);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    #pad1.SetBorderMode(0)
    #pad1.SetBorderSize(0)
    #pad1.SetFrameLineWidth(0)
    #pad1.SetFrameBorderMode(0)
    pad1.SetTopMargin(0.1);
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_topSel[i])
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(2)
    TTSemileptonic.SetFillColor(0)
    
    TTDileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[2], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[2], nevt=nevts[2]))
    TTDileptonic.SetLineColor(ROOT.kRed-9)
    TTDileptonic.SetFillColor(0)
    TTDileptonic.SetLineWidth(2)
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(0)
    WJets.SetLineWidth(2)
    
    SingleTop = WJets.Clone()
    SingleTop.Reset()
    for j in range(4,9):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(0)
    SingleTop.SetLineWidth(2)
    
    VV = WJets.Clone()
    VV.Reset()
    for j in range(9,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(0)
    VV.SetLineWidth(2)
    
    QCD = WJets.Clone()
    QCD.Reset()
    for j in range(12,18):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(0)
    QCD.SetLineWidth(2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTSemileptonic.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTDileptonic)
    MC.Add(TTSemileptonic)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_topSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    dummy.GetYaxis().SetRangeUser(-2, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    #c[i].Update()
    #MC.Draw("histsame e")
    Data_SingleMuon_Combined = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathData, xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuon_Combined.SetMarkerSize(1.5)
    Data_SingleMuon_Combined.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuon_Combined.SetMarkerStyle(8)
    Data_SingleMuon_Combined.SetLineColor(1)
    Data_SingleMuon_Combined.SetFillColor(0)
    Data_SingleMuon_Combined.Draw("AE same")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1.,0.207, -1)
    pad2.SetTopMargin(0)
    pad2.SetGrid()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    ratio.GetXaxis().SetTitle(Histnames_topSel[i][2])
    ratio.Add(Data_SingleMuon_Combined)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    mc_rt.Add(TTSemileptonic)
    mc_rt.Add(TTDileptonic)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","l")
    l[i].AddEntry(TTDileptonic,"t#bar{t} dileptonic","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuon_Combined,"Data UL17+UL18","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/Wtop/topSel/Basic/all/"+Hists_topSel[i]+"_powheg_pythia8_UL17-18_NoTopReWt.pdf")
    c[i].Close()

0 recoJet_pt_nom_topSel ../test/Samples/SingleMuonAll_UL17-18.root [30, 0, 1500] 101288.44200000001 301.64 63770862770.80198
P_{t}/GeV
1 recoJet_mass_nom_topSel ../test/Samples/SingleMuonAll_UL17-18.root [12, 130, 250] 101288.44200000001 301.64 63770862770.80198
Softdrop Mass/GeV
2 recoJet_tau21_nom_topSel ../test/Samples/SingleMuonAll_UL17-18.root [20, 0, 1] 101288.44200000001 301.64 63770862770.80198
#tau_{21}
3 recoJet_tau32_nom_topSel ../test/Samples/SingleMuonAll_UL17-18.root [20, 0, 1] 101288.44200000001 301.64 63770862770.80198
#tau_{32}
4 recoJet_tau_0p5_1_nom_topSel ../test/Samples/SingleMuonAll_UL17-18.root [30, 0, 0.9] 101288.44200000001 301.64 63770862770.80198
#tau_{1}^{(0.5)}
5 recoJet_tau_1_1_nom_topSel ../test/Samples/SingleMuonAll_UL17-18.root [35, 0, 0.7] 101288.44200000001 301.64 63770862770.80198
#tau_{1}^{(1)}
6 recoJet_tau_2_1_nom_topSel ../test/Samples/SingleMuonAll_UL17-18.root [20, 0, 0.4] 101288.44200000001 301.64 63770862770.80198
#tau_{1}^{(2)}
7 recoJet_tau

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/all/recoJet_pt_nom_topSel_powheg_pythia8_UL17-18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/all/recoJet_mass_nom_topSel_powheg_pythia8_UL17-18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/all/recoJet_tau21_nom_topSel_powheg_pythia8_UL17-18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/all/recoJet_tau32_nom_topSel_powheg_pythia8_UL17-18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/all/recoJet_tau_0p5_1_nom_topSel_powheg_pythia8_UL17-18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/all/recoJet_tau_1_1_nom_topSel_powheg_pythia8_UL17-18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results

### 2017

In [46]:
lumi=41472.052


In [47]:
xsecs = list(dictSamples[s]['XS'] for s in list(dictSamples.keys())[1:] )
nevts = list(dictSamples[s]['2017']['nGenWeights'] for s in list(dictSamples.keys())[1:] )

fpathsMC = list('../test/Samples/'+dictSamples[s]['2017']['skimmerHisto'] for s in list(dictSamples.keys())[1:] )
fpathData = '../test/Samples/'+dictSamples['SingleMuon']['2017']['skimmerHisto']
for i in range(0,len(xsecs)):
    print (i, xsecs[i], nevts[i], fpathsMC[i])
print (fpathData)

0 753.5 286629789811.7677 ../test/Samples/TTJets_amcatnloFXFX-pythia8_2017UL.root
1 301.64 32392787477.907993 ../test/Samples/TTToSemileptonic_powheg_pythia8_2017UL_NoTopReWt.root
2 72.83 4759777104.469413 ../test/Samples/TTTo2L2Nu_powheg_pythia8_2017UL.root
3 53710.0 9949891586212.455 ../test/Samples/WJetsToLNu_madgraphMLM_pythia8_2017UL.root
4 3.549 70439807.475972 ../test/Samples/ST_s-channel_amcatnlo_pythia8_2017UL.root
5 119.7 656310119.274 ../test/Samples/ST_t-channel_top_powheg_pythia8_2017UL.root
6 71.74 303960792.4356 ../test/Samples/ST_t-channel_antitop_powheg_pythia8_2017UL.root
7 32.45 331239851.9677 ../test/Samples/ST_tW_top_powheg_pythia8_2017UL.root
8 32.51 303960792.4356 ../test/Samples/ST_tW_antitop_powheg_pythia8_2017UL.root
9 75.6 15634116.199514 ../test/Samples/WW_pythia8_2017UL.root
10 12.1 2706000.0 ../test/Samples/ZZ_pythia8_2017UL.root
11 27.51 7889000.0 ../test/Samples/WZ_pythia8_2017UL.root
12 7014.0 36027673.0 ../test/Samples/QCD_Muen_Pt_170to300_pythia8_2017

In [48]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.14

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [49]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 750; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.207, 1., 1., -1);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    #pad1.SetBorderMode(0)
    #pad1.SetBorderSize(0)
    #pad1.SetFrameLineWidth(0)
    #pad1.SetFrameBorderMode(0)
    pad1.SetTopMargin(0.1);
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_topSel[i])
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(2)
    TTSemileptonic.SetFillColor(0)
    
    TTDileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[2], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[2], nevt=nevts[2]))
    TTDileptonic.SetLineColor(ROOT.kRed-9)
    TTDileptonic.SetFillColor(0)
    TTDileptonic.SetLineWidth(2)
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(0)
    WJets.SetLineWidth(2)
    
    SingleTop = WJets.Clone()
    SingleTop.Reset()
    for j in range(4,9):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(0)
    SingleTop.SetLineWidth(2)
    
    VV = WJets.Clone()
    VV.Reset()
    for j in range(9,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(0)
    VV.SetLineWidth(2)
    
    QCD = WJets.Clone()
    QCD.Reset()
    for j in range(12,18):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(0)
    QCD.SetLineWidth(2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTSemileptonic.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTDileptonic)
    MC.Add(TTSemileptonic)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_topSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    dummy.GetYaxis().SetRangeUser(-2, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    #c[i].Update()
    #MC.Draw("histsame e")
    Data_SingleMuon_Combined = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathData, xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuon_Combined.SetMarkerSize(1.5)
    Data_SingleMuon_Combined.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuon_Combined.SetMarkerStyle(8)
    Data_SingleMuon_Combined.SetLineColor(1)
    Data_SingleMuon_Combined.SetFillColor(0)
    Data_SingleMuon_Combined.Draw("AE same")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1.,0.207, -1)
    pad2.SetTopMargin(0)
    pad2.SetGrid()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    ratio.GetXaxis().SetTitle(Histnames_topSel[i][2])
    ratio.Add(Data_SingleMuon_Combined)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    mc_rt.Add(TTSemileptonic)
    mc_rt.Add(TTDileptonic)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","l")
    l[i].AddEntry(TTDileptonic,"t#bar{t} dileptonic","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuon_Combined,"Data UL17","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/Wtop/topSel/Basic/2017/"+Hists_WSel[i]+"_powheg_pythia8_UL17_NoTopReWt.pdf")
    c[i].Close()
    c[i].Close()

0 recoJet_pt_nom_topSel ../test/Samples/SingleMuonAll_2017UL.root [30, 0, 1500] 41472.052 301.64 32392787477.907993
P_{t}/GeV
1 recoJet_mass_nom_topSel ../test/Samples/SingleMuonAll_2017UL.root [12, 130, 250] 41472.052 301.64 32392787477.907993
Softdrop Mass/GeV
2 recoJet_tau21_nom_topSel ../test/Samples/SingleMuonAll_2017UL.root [20, 0, 1] 41472.052 301.64 32392787477.907993
#tau_{21}
3 recoJet_tau32_nom_topSel ../test/Samples/SingleMuonAll_2017UL.root [20, 0, 1] 41472.052 301.64 32392787477.907993
#tau_{32}
4 recoJet_tau_0p5_1_nom_topSel ../test/Samples/SingleMuonAll_2017UL.root [30, 0, 0.9] 41472.052 301.64 32392787477.907993
#tau_{1}^{(0.5)}
5 recoJet_tau_1_1_nom_topSel ../test/Samples/SingleMuonAll_2017UL.root [35, 0, 0.7] 41472.052 301.64 32392787477.907993
#tau_{1}^{(1)}
6 recoJet_tau_2_1_nom_topSel ../test/Samples/SingleMuonAll_2017UL.root [20, 0, 0.4] 41472.052 301.64 32392787477.907993
#tau_{1}^{(2)}
7 recoJet_tau_0p5_2_nom_topSel ../test/Samples/SingleMuonAll_2017UL.root [20

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2017/recoJet_pt_nom_WSel_powheg_pythia8_UL17_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2017/recoJet_mass_nom_WSel_powheg_pythia8_UL17_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2017/recoJet_tau21_nom_WSel_powheg_pythia8_UL17_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2017/recoJet_tau32_nom_WSel_powheg_pythia8_UL17_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2017/recoJet_tau_0p5_1_nom_WSel_powheg_pythia8_UL17_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2017/recoJet_tau_1_1_nom_WSel_powheg_pythia8_UL17_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic

### 2018

In [50]:
lumi=59816.39


In [51]:
xsecs = list(dictSamples[s]['XS'] for s in list(dictSamples.keys())[1:] )
nevts = list(dictSamples[s]['2018']['nGenWeights'] for s in list(dictSamples.keys())[1:] )

fpathsMC = list('../test/Samples/'+dictSamples[s]['2018']['skimmerHisto'] for s in list(dictSamples.keys())[1:] )
fpathData = '../test/Samples/'+dictSamples['SingleMuon']['2018']['skimmerHisto']
for i in range(0,len(xsecs)):
    print (i, xsecs[i], nevts[i], fpathsMC[i])
print (fpathData)

0 753.5 297902725633.38257 ../test/Samples/TTJets_amcatnloFXFX-pythia8_2018UL.root
1 301.64 31378075292.893986 ../test/Samples/TTToSemileptonic_powheg_pythia8_2018UL_NoTopReWt.root
2 72.83 5063754431.9676 ../test/Samples/TTTo2L2Nu_powheg_pythia8_2018UL.root
3 53710.0 10484986870276.357 ../test/Samples/WJetsToLNu_madgraphMLM_pythia8_2018UL.root
4 3.549 64014403.924644 ../test/Samples/ST_s-channel_amcatnlo_pythia8_2018UL.root
5 119.7 683747124.99 ../test/Samples/ST_t-channel_top_powheg_pythia8_2018UL.root
6 71.74 271219283.652 ../test/Samples/ST_t-channel_antitop_powheg_pythia8_2018UL.root
7 32.45 328321574.253 ../test/Samples/ST_tW_top_powheg_pythia8_2018UL.root
8 32.51 296655487.5576 ../test/Samples/ST_tW_antitop_powheg_pythia8_2018UL.root
9 75.6 7959266.140092 ../test/Samples/WW_pythia8_2018UL.root
10 12.1 2000000.0 ../test/Samples/ZZ_pythia8_2018UL.root
11 27.51 3976000.0 ../test/Samples/WZ_pythia8_2018UL.root
12 7014.0 35868369.0 ../test/Samples/QCD_Muen_Pt_170to300_pythia8_2018UL.r

In [52]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.14

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [53]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 750; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.207, 1., 1., -1);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    #pad1.SetBorderMode(0)
    #pad1.SetBorderSize(0)
    #pad1.SetFrameLineWidth(0)
    #pad1.SetFrameBorderMode(0)
    pad1.SetTopMargin(0.1);
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_topSel[i])
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(2)
    TTSemileptonic.SetFillColor(0)
    
    TTDileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[2], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[2], nevt=nevts[2]))
    TTDileptonic.SetLineColor(ROOT.kRed-9)
    TTDileptonic.SetFillColor(0)
    TTDileptonic.SetLineWidth(2)
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(0)
    WJets.SetLineWidth(2)
    
    SingleTop = WJets.Clone()
    SingleTop.Reset()
    for j in range(4,9):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(0)
    SingleTop.SetLineWidth(2)
    
    VV = WJets.Clone()
    VV.Reset()
    for j in range(9,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(0)
    VV.SetLineWidth(2)
    
    QCD = WJets.Clone()
    QCD.Reset()
    for j in range(12,18):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(0)
    QCD.SetLineWidth(2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTSemileptonic.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTDileptonic)
    MC.Add(TTSemileptonic)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_topSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    dummy.GetYaxis().SetRangeUser(-2, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    #c[i].Update()
    #MC.Draw("histsame e")
    Data_SingleMuon_Combined = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathData, xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuon_Combined.SetMarkerSize(1.5)
    Data_SingleMuon_Combined.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuon_Combined.SetMarkerStyle(8)
    Data_SingleMuon_Combined.SetLineColor(1)
    Data_SingleMuon_Combined.SetFillColor(0)
    Data_SingleMuon_Combined.Draw("AE same")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1.,0.207, -1)
    pad2.SetTopMargin(0)
    pad2.SetGrid()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    ratio.GetXaxis().SetTitle(Histnames_topSel[i][2])
    ratio.Add(Data_SingleMuon_Combined)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    mc_rt.Add(TTSemileptonic)
    mc_rt.Add(TTDileptonic)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","l")
    l[i].AddEntry(TTDileptonic,"t#bar{t} dileptonic","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuon_Combined,"Data UL18","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/Wtop/topSel/Basic/2018/"+Hists_WSel[i]+"_powheg_pythia8_UL18_NoTopReWt.pdf")
    c[i].Close()
    c[i].Close()

0 recoJet_pt_nom_topSel ../test/Samples/SingleMuonAll_2018UL.root [30, 0, 1500] 59816.39 301.64 31378075292.893986
P_{t}/GeV
1 recoJet_mass_nom_topSel ../test/Samples/SingleMuonAll_2018UL.root [12, 130, 250] 59816.39 301.64 31378075292.893986
Softdrop Mass/GeV
2 recoJet_tau21_nom_topSel ../test/Samples/SingleMuonAll_2018UL.root [20, 0, 1] 59816.39 301.64 31378075292.893986
#tau_{21}
3 recoJet_tau32_nom_topSel ../test/Samples/SingleMuonAll_2018UL.root [20, 0, 1] 59816.39 301.64 31378075292.893986
#tau_{32}
4 recoJet_tau_0p5_1_nom_topSel ../test/Samples/SingleMuonAll_2018UL.root [30, 0, 0.9] 59816.39 301.64 31378075292.893986
#tau_{1}^{(0.5)}
5 recoJet_tau_1_1_nom_topSel ../test/Samples/SingleMuonAll_2018UL.root [35, 0, 0.7] 59816.39 301.64 31378075292.893986
#tau_{1}^{(1)}
6 recoJet_tau_2_1_nom_topSel ../test/Samples/SingleMuonAll_2018UL.root [20, 0, 0.4] 59816.39 301.64 31378075292.893986
#tau_{1}^{(2)}
7 recoJet_tau_0p5_2_nom_topSel ../test/Samples/SingleMuonAll_2018UL.root [20, 0, 0.

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2018/recoJet_pt_nom_WSel_powheg_pythia8_UL18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2018/recoJet_mass_nom_WSel_powheg_pythia8_UL18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2018/recoJet_tau21_nom_WSel_powheg_pythia8_UL18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2018/recoJet_tau32_nom_WSel_powheg_pythia8_UL18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2018/recoJet_tau_0p5_1_nom_WSel_powheg_pythia8_UL18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic/2018/recoJet_tau_1_1_nom_WSel_powheg_pythia8_UL18_NoTopReWt.pdf has been created
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Wtop/topSel/Basic

### Resolution plots


In [ ]:
Hists_WSel=np.array([
    
    'resolJet_tau_0p5_1_WSel', 'resolJet_tau_1_1_WSel', 'resolJet_tau_2_1_WSel',
    'resolJet_tau_0p5_2_WSel', 'resolJet_tau_1_2_WSel', 'resolJet_tau_2_2_WSel',
    'resolJet_tau_0p5_3_WSel', 'resolJet_tau_1_3_WSel', 'resolJet_tau_2_3_WSel',
    'resolJet_tau_0p5_4_WSel', 'resolJet_tau_1_4_WSel', 'resolJet_tau_2_4_WSel',
    'resolJet_tau21_WSel', 'resolJet_tau32_WSel',
])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events','#resol tau_{1}^{(0.5)}','resol #tau_{1}^{(0.5)}'],['Events','resol #tau_{1}^{(1)}','resol #tau_{1}^{(1)}'],['Events','resol #tau_{1}^{(2)}','resol #tau_{1}^{(2)}'],
    ['Events','#resol tau_{2}^{(0.5)}','resol #tau_{2}^{(0.5)}'],['Events','resol #tau_{2}^{(1)}','resol #tau_{2}^{(1)}'],['Events','resol #tau_{2}^{(2)}','resol #tau_{2}^{(2)}'],
    ['Events','#resol tau_{3}^{(0.5)}','resol #tau_{3}^{(0.5)}'],['Events','resol #tau_{3}^{(1)}','resol #tau_{3}^{(1)}'],['Events','resol #tau_{3}^{(2)}','resol #tau_{3}^{(2)}'],
    ['Events','#resol tau_{4}^{(0.5)}','resol #tau_{4}^{(0.5)}'],['Events','resol #tau_{4}^{(1)}','resol #tau_{4}^{(1)}'],['Events','resol #tau_{4}^{(2)}','resol #tau_{4}^{(2)}'],
    ['Events','resol #tau_{2,1}^{(1)}','resol #tau_{2,1}^{(1)}'],['Events','resol #tau_{3,2}^{(1)}','resol #tau_{3,2}^{(1)}'],
    ])
Range_WSel=[
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.]
    
    
]

Hists_topSel=np.array([
    
    'resolJet_tau_0p5_1_topSel', 'resolJet_tau_1_1_topSel', 'resolJet_tau_2_1_topSel',
    'resolJet_tau_0p5_2_topSel', 'resolJet_tau_1_2_topSel', 'resolJet_tau_2_2_topSel',
    'resolJet_tau_0p5_3_topSel', 'resolJet_tau_1_3_topSel', 'resolJet_tau_2_3_topSel',
    'resolJet_tau_0p5_4_topSel', 'resolJet_tau_1_4_topSel', 'resolJet_tau_2_4_topSel',
    'resolJet_tau21_topSel', 'resolJet_tau32_topSel',
])
#Objects_topSel=np.concatenate((Objects1_topSel,Objects3_topSel),axis=0)

Histnames_topSel=np.array([
    ['Events','#resol tau_{1}^{(0.5)}','resol #tau_{1}^{(0.5)}'],['Events','resol #tau_{1}^{(1)}','resol #tau_{1}^{(1)}'],['Events','resol #tau_{1}^{(2)}','resol #tau_{1}^{(2)}'],
    ['Events','#resol tau_{2}^{(0.5)}','resol #tau_{2}^{(0.5)}'],['Events','resol #tau_{2}^{(1)}','resol #tau_{2}^{(1)}'],['Events','resol #tau_{2}^{(2)}','resol #tau_{2}^{(2)}'],
    ['Events','#resol tau_{3}^{(0.5)}','resol #tau_{3}^{(0.5)}'],['Events','resol #tau_{3}^{(1)}','resol #tau_{3}^{(1)}'],['Events','resol #tau_{3}^{(2)}','resol #tau_{3}^{(2)}'],
    ['Events','#resol tau_{4}^{(0.5)}','resol #tau_{4}^{(0.5)}'],['Events','resol #tau_{4}^{(1)}','resol #tau_{4}^{(1)}'],['Events','resol #tau_{4}^{(2)}','resol #tau_{4}^{(2)}'],
    ['Events','resol #tau_{2,1}^{(1)}','resol #tau_{2,1}^{(1)}'],['Events','resol #tau_{3,2}^{(1)}','resol #tau_{3,2}^{(1)}'],
    ])
Range_topSel=[
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.]
    
    
]
print (len(Hists_topSel),len(Histnames_topSel),len(Range_topSel))
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

In [ ]:
print (Histnames_WSel[0][0]
      )

###  all

In [ ]:
lumi=41472.052+59816.39


In [ ]:
xsecs = list(dictSamples[s]['XS'] for s in list(dictSamples.keys())[1:] )
nevts = list(dictSamples[s]['all']['nGenWeights'] for s in list(dictSamples.keys())[1:] )

fpathsMC = list('../test/Samples/'+dictSamples[s]['all']['skimmerHisto'] for s in list(dictSamples.keys())[1:] )
fpathData = '../test/Samples/'+dictSamples['SingleMuon']['all']['skimmerHisto']
for i in range(0,len(xsecs)):
    print (i, xsecs[i], nevts[i], fpathsMC[i])
print (fpathData)

In [ ]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.11

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [ ]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 500; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.45
y1_l = 0.9

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    #c[i].SetBorderMode(0)
    #c[i].SetBorderSize(0)
    #c[i].SetFrameFillStyle(0)
    #c[i].SetFrameBorderMode(0)
    #c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    #c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    #c[i].SetTickx(0)
    #c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    #TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(2)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
      
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    #TTSemileptonic.GetYaxis().SetRangeUser(-150.,TTSemileptonic.GetMaximum()*2.2)
    TTSemileptonic.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    #TTJets.Rebin(2)
    TTSemileptonic.Scale(1/TTSemileptonic.Integral())
    binWidth = TTSemileptonic.GetBinWidth(1)

    TTSemileptonic.GetYaxis().SetTitle('Normalized / '+str(binWidth))

    TTSemileptonic.Draw("PE")
    #TTJets.Draw("ep same")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    #l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextSize(0.03)    
    #legend.SetTextSize(0.05)    
    #l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    CMS_lumi.CMS_lumi(c[i], 4, 0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/WSel/all/Resolution/"+Hists_WSel[i]+"UL17-18_NoTopReWt.pdf")
    c[i].Close()

In [ ]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 500; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.45
y1_l = 0.9

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    #c[i].SetBorderMode(0)
    #c[i].SetBorderSize(0)
    #c[i].SetFrameFillStyle(0)
    #c[i].SetFrameBorderMode(0)
    #c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    #c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    #c[i].SetTickx(0)
    #c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    #TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(2)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
      
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    #TTSemileptonic.GetYaxis().SetRangeUser(-150.,TTSemileptonic.GetMaximum()*2.2)
    TTSemileptonic.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    #TTJets.Rebin(2)
    TTSemileptonic.Scale(1/TTSemileptonic.Integral())
    binWidth = TTSemileptonic.GetBinWidth(1)

    TTSemileptonic.GetYaxis().SetTitle('Normalized / '+str(binWidth))

    TTSemileptonic.Draw("PE")
    #TTJets.Draw("ep same")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    #l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextSize(0.03)    
    #legend.SetTextSize(0.05)    
    #l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    CMS_lumi.CMS_lumi(c[i], 4, 0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/topSel/all/Resolution/"+Hists_topSel[i]+"UL17-18_NoTopReWt.pdf")
    c[i].Close()

### 2017

In [ ]:
lumi=41472.052


In [ ]:
xsecs = list(dictSamples[s]['XS'] for s in list(dictSamples.keys())[1:] )
nevts = list(dictSamples[s]['2017']['nGenWeights'] for s in list(dictSamples.keys())[1:] )

fpathsMC = list('../test/Samples/'+dictSamples[s]['2017']['skimmerHisto'] for s in list(dictSamples.keys())[1:] )
fpathData = '../test/Samples/'+dictSamples['SingleMuon']['2017']['skimmerHisto']
for i in range(0,len(xsecs)):
    print (i, xsecs[i], nevts[i], fpathsMC[i])
print (fpathData)

In [ ]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.11

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [ ]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 500; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.45
y1_l = 0.9

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    #c[i].SetBorderMode(0)
    #c[i].SetBorderSize(0)
    #c[i].SetFrameFillStyle(0)
    #c[i].SetFrameBorderMode(0)
    #c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    #c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    #c[i].SetTickx(0)
    #c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    #TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(2)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
      
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    #TTSemileptonic.GetYaxis().SetRangeUser(-150.,TTSemileptonic.GetMaximum()*2.2)
    TTSemileptonic.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    #TTJets.Rebin(2)
    TTSemileptonic.Scale(1/TTSemileptonic.Integral())
    binWidth = TTSemileptonic.GetBinWidth(1)

    TTSemileptonic.GetYaxis().SetTitle('Normalized / '+str(binWidth))

    TTSemileptonic.Draw("PE")
    #TTJets.Draw("ep same")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    #l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextSize(0.03)    
    #legend.SetTextSize(0.05)    
    #l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    CMS_lumi.CMS_lumi(c[i], 4, 0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/WSel/2017/Resolution/"+Hists_WSel[i]+"UL17_NoTopReWt.pdf")
    c[i].Close()

In [ ]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 500; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.45
y1_l = 0.9

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    #c[i].SetBorderMode(0)
    #c[i].SetBorderSize(0)
    #c[i].SetFrameFillStyle(0)
    #c[i].SetFrameBorderMode(0)
    #c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    #c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    #c[i].SetTickx(0)
    #c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    #TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(2)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
      
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    #TTSemileptonic.GetYaxis().SetRangeUser(-150.,TTSemileptonic.GetMaximum()*2.2)
    TTSemileptonic.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    #TTJets.Rebin(2)
    TTSemileptonic.Scale(1/TTSemileptonic.Integral())
    binWidth = TTSemileptonic.GetBinWidth(1)

    TTSemileptonic.GetYaxis().SetTitle('Normalized / '+str(binWidth))

    TTSemileptonic.Draw("PE")
    #TTJets.Draw("ep same")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    #l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextSize(0.03)    
    #legend.SetTextSize(0.05)    
    #l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    CMS_lumi.CMS_lumi(c[i], 4, 0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/topSel/2017/Resolution/"+Hists_topSel[i]+"UL17_NoTopReWt.pdf")
    c[i].Close()

###  2018

In [ ]:
lumi=59816.39


In [ ]:
xsecs = list(dictSamples[s]['XS'] for s in list(dictSamples.keys())[1:] )
nevts = list(dictSamples[s]['2018']['nGenWeights'] for s in list(dictSamples.keys())[1:] )

fpathsMC = list('../test/Samples/'+dictSamples[s]['2018']['skimmerHisto'] for s in list(dictSamples.keys())[1:] )
fpathData = '../test/Samples/'+dictSamples['SingleMuon']['2018']['skimmerHisto']
for i in range(0,len(xsecs)):
    print (i, xsecs[i], nevts[i], fpathsMC[i])
print (fpathData)

In [ ]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.11

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [ ]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 500; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.45
y1_l = 0.9

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    #c[i].SetBorderMode(0)
    #c[i].SetBorderSize(0)
    #c[i].SetFrameFillStyle(0)
    #c[i].SetFrameBorderMode(0)
    #c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    #c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    #c[i].SetTickx(0)
    #c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    #TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(2)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
      
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    #TTSemileptonic.GetYaxis().SetRangeUser(-150.,TTSemileptonic.GetMaximum()*2.2)
    TTSemileptonic.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    #TTJets.Rebin(2)
    TTSemileptonic.Scale(1/TTSemileptonic.Integral())
    binWidth = TTSemileptonic.GetBinWidth(1)

    TTSemileptonic.GetYaxis().SetTitle('Normalized / '+str(binWidth))

    TTSemileptonic.Draw("PE")
    #TTJets.Draw("ep same")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    #l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextSize(0.03)    
    #legend.SetTextSize(0.05)    
    #l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    CMS_lumi.CMS_lumi(c[i], 4, 0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/WSel/2018/Resolution/"+Hists_WSel[i]+"UL18_NoTopReWt.pdf")
    c[i].Close()

In [ ]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 500; 
W_ref = 750; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.45
y1_l = 0.9

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],10,10,W,H)
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    c[i].SetFillColor(0)
    #c[i].SetBorderMode(0)
    #c[i].SetBorderSize(0)
    #c[i].SetFrameFillStyle(0)
    #c[i].SetFrameBorderMode(0)
    #c[i].SetFrameLineWidth(0)
    #c[i].SetLeftMargin( L/W )
    #c[i].SetRightMargin( R/W )
    #c[i].SetTopMargin( T/H )
    #c[i].SetBottomMargin( B/H )
    #c[i].SetTickx(0)
    #c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    #TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(2)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
      
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    #TTSemileptonic.GetYaxis().SetRangeUser(-150.,TTSemileptonic.GetMaximum()*2.2)
    TTSemileptonic.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    #TTJets.Rebin(2)
    TTSemileptonic.Scale(1/TTSemileptonic.Integral())
    binWidth = TTSemileptonic.GetBinWidth(1)

    TTSemileptonic.GetYaxis().SetTitle('Normalized / '+str(binWidth))

    TTSemileptonic.Draw("PE")
    #TTJets.Draw("ep same")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    #l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextSize(0.03)    
    #legend.SetTextSize(0.05)    
    #l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    CMS_lumi.CMS_lumi(c[i], 4, 0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("../test/Results_Local/topSel/2018/Resolution/"+Hists_topSel[i]+"UL18_NoTopReWt.pdf")
    c[i].Close()